<a href="https://colab.research.google.com/github/armeh429/A-journey-to-learn-Python-modules/blob/main/pycoingecko_with_candlechart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduction**

An API lets two pieces of software talk to each other. Just like a function, you don’t have to know how the API works only its inputs and outputs. An essential type of API is a REST API that allows you to access resources via the internet. In this lab, we will review the Pandas Library in the context of an API, we will also review a basic REST API

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

#**Pandas is an API** 

Pandas is actually set of software components , much of which is not even written in Python.

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
import time
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

You create a dictionary, this is just data.

In [4]:
dict_={'a':[11,21,31],'b':[12,22,32]}

When you create a Pandas object with the Dataframe constructor in API lingo, this is an "instance". The data in the dictionary is passed along to the pandas API. You then use the dataframe to communicate with the API.

In [5]:
df=pd.DataFrame(dict_)
type(df)

pandas.core.frame.DataFrame

When you call the method head the dataframe communicates with the API displaying the first few rows of the dataframe.

In [6]:
df.head()

,a,b
0,11,12
1,21,22
2,31,32


When you call the method mean,the API will calculate the mean and return the value.

In [7]:
df.mean()

a    21.0
b    22.0
dtype: float64

#**REST APIs¶**

Rest API’s function by sending a request, the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [8]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [9]:
type(bitcoin_data)

dict

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [10]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1632690026503, 43673.57143121867],
 [1632693798930, 43644.345877313324],
 [1632697649451, 43087.02144443022],
 [1632700854823, 43336.861233242744],
 [1632704403273, 43890.92814242138]]

Finally lets turn this data into a Pandas DataFrame.

In [11]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [12]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [13]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally we are now ready to use plotly to create our Candlestick Chart.

In [14]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

# 4 hour time frame

In [15]:
data['time'] = data['TimeStamp'].apply(lambda d: time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(d/1000)))
data['hour'] = data.time.astype('datetime64').apply(lambda i:i.hour)
data[0:24]

,TimeStamp,Price,date,time,hour
0,1632690026503,43673.571431,2021-09-27,2021-09-26 21:00:26,21
1,1632693798930,43644.345877,2021-09-27,2021-09-26 22:03:18,22
2,1632697649451,43087.021444,2021-09-27,2021-09-26 23:07:29,23
3,1632700854823,43336.861233,2021-09-27,2021-09-27 00:00:54,0
4,1632704403273,43890.928142,2021-09-27,2021-09-27 01:00:03,1
5,1632708128535,44346.563424,2021-09-27,2021-09-27 02:02:08,2
6,1632711814234,44262.685836,2021-09-27,2021-09-27 03:03:34,3
7,1632715388846,44034.181620,2021-09-27,2021-09-27 04:03:08,4
8,1632718923893,44032.649990,2021-09-27,2021-09-27 05:02:03,5
9,1632722467244,44136.446182,2021-09-27,2021-09-27 06:01:07,6


In [16]:
#function to group H4 data
def H4date(x):
  if 0<=x<=3: return 1
  elif 4<=x<=7: return 2
  elif 8<=x<=11: return 3
  elif 12<=x<=15: return 4
  elif 16<=x<=19: return 5
  elif 20<=x<=23: return 6

data['H4'] = data.hour.apply(H4date)
data['H4_date'] = data.date.astype(str)+'_'+data.H4.astype(str)
data

,TimeStamp,Price,date,time,hour,H4,H4_date
0,1632690026503,43673.571431,2021-09-27,2021-09-26 21:00:26,21,6,2021-09-27_6
1,1632693798930,43644.345877,2021-09-27,2021-09-26 22:03:18,22,6,2021-09-27_6
2,1632697649451,43087.021444,2021-09-27,2021-09-26 23:07:29,23,6,2021-09-27_6
3,1632700854823,43336.861233,2021-09-27,2021-09-27 00:00:54,0,1,2021-09-27_1
4,1632704403273,43890.928142,2021-09-27,2021-09-27 01:00:03,1,1,2021-09-27_1
...,...,...,...,...,...,...,...
717,1635267979688,62191.380597,2021-10-26,2021-10-26 17:06:19,17,5,2021-10-26_5
718,1635271319600,62706.642851,2021-10-26,2021-10-26 18:01:59,18,5,2021-10-26_5
719,1635274825816,62534.566193,2021-10-26,2021-10-26 19:00:25,19,5,2021-10-26_5
720,1635278547030,62281.467808,2021-10-26,2021-10-26 20:02:27,20,6,2021-10-26_6


In [17]:
candlestick_data_H4 = data.groupby(data.H4_date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

In [18]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data_H4['H4_date'],
                open=candlestick_data_H4['Price']['first'], 
                high=candlestick_data_H4['Price']['max'],
                low=candlestick_data_H4['Price']['min'], 
                close=candlestick_data_H4['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()